In [1]:
import pandas as pd # Data handling 
import numpy as np

import tensorflow as tf # Neural networks  
from tensorflow import keras
from tensorflow.keras import layers

import plotly.graph_objects as go # Visualization
import plotly.express as px

import sklearn as sk # Stats / ML
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV, TimeSeriesSplit
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import impute

from xgboost import XGBRegressor # Efficient Gradient Boosting

In [3]:
df_train = pd.read_csv("Data/train_imputed.csv")
df_test = pd.read_csv("Data/test_imputed.csv")

display(df_train.head())
display(df_test.head())

,year,weekofyear,city,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,1990,18,1,1990-04-30,0.122600,0.103725,0.198483,0.177617,297.572857,297.742857,...,32.00,73.365714,12.42,2.628571,25.442857,6.900000,29.4,20.0,16.0,4.0
1,1990,19,1,1990-05-07,0.169900,0.142175,0.162357,0.155486,298.211429,298.442857,...,17.94,77.368571,22.82,2.371429,26.714286,6.371429,31.7,22.2,8.6,5.0
2,1990,20,1,1990-05-14,0.032250,0.172967,0.157200,0.170843,298.781429,298.878571,...,26.10,82.052857,34.54,2.300000,26.714286,6.485714,32.2,22.8,41.4,4.0
3,1990,21,1,1990-05-21,0.128633,0.245067,0.227557,0.235886,298.987143,299.228571,...,13.90,80.337143,15.36,2.428571,27.471429,6.771429,33.3,23.3,4.0,3.0
4,1990,22,1,1990-05-28,0.196200,0.262200,0.251200,0.247340,299.518571,299.664286,...,12.20,80.460000,7.52,3.014286,28.942857,9.371429,35.0,23.9,5.8,6.0


,year,weekofyear,city,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,2008,18,1,2008-04-29,-0.018900,-0.018900,0.102729,0.091200,298.492857,298.550000,...,301.1,25.37,78.781429,78.60,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,2008,19,1,2008-05-06,-0.018000,-0.012400,0.082043,0.072314,298.475714,298.557143,...,300.8,21.83,78.230000,12.56,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,2008,20,1,2008-05-13,-0.001500,-0.004324,0.151083,0.091529,299.455714,299.357143,...,302.2,4.12,78.270000,3.66,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,2008,21,1,2008-05-20,0.202833,-0.019867,0.124329,0.125686,299.690000,299.728571,...,303.0,2.20,73.015714,0.00,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,2008,22,1,2008-05-27,0.056800,0.039833,0.062267,0.075914,299.780000,299.671429,...,302.3,4.36,74.084286,0.76,3.542857,27.614286,7.085714,33.3,23.3,84.1
